# <font color="blue"> LESSON 3: Tidy Data in Python with Pandas </font>


# Lesson Goals
 To demonstrate how any dataset can be structured in a standardized way prior to analysis, such that it facilitates the analysis.
 
 Data cleaning is one the most frequent tasks in data science. No matter what kind of data you are dealing with or what kind of analysis you are performing, you will have to clean the data at some point. Tidying your data in a standard format makes things easier down the road. You can reuse a standard set of tools across your different analysis.
 
This  lesson will summarize some tidying examples and will demonstrate how to do so using Python pandas.

# Defining tidy data:

[Tidy data](reports/tidy-data.pdf) has the following attributes:

<ul>
<li>Each variable forms a column and contains values</li>
<li>Each observation forms a row</li>
</ul>

Below we remind you the definitions of variable and value as well as defining observation.  The examples assume we are looking at a patient database, containing information on the patients and their treatments:
<ul>
<li>Variable = A "box" storing a value. Examples: Height, weight, name, treatment, etc.</li>
<li>Value = The actual measurement or attribute stored in the variable. Examples: 152 cm, 80 kg, Jane Doe, pill, etc.</li>
<li>Observation = All values pertaining to a single entity. Examples: each patient.</li>
</ul>

Let's look at some examples of a <em>messy</em> DataFrame and a <em>tidy</em> DataFrame.

# An example of a messy dataset:

Problem: We want each variable to form a column.  Below we have the variable <em>Treatment</em> forming two columns.

<img src=images/messy_data.png>



# An example of a tidy dataset:

Solution: Convert <em>Treatment</em> to a single column with values a or b.

<img src=images/tidy_data.png>


These are the four types of messy datasets we’ll tackle:
<ol>
<li>Column headers are values, not variable names.</li>
<li>Multiple variables are stored in one column.</li>
<li>Variables are stored in both rows and columns.</li>
<li>A single observational unit is stored in multiple tables.</li>
</ol>

# 1. Column headers are values, not variable names

Pew Research Center Dataset

This dataset explores the relationship between income and religion.

Problem: The columns headers are composed of the possible income values.

Import all the packages we need

In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

In [2]:
df = pd.read_csv("./data/pew-raw.csv")
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35


A tidy version of this dataset is one in which the income values would not be columns headers but rather values in an income column. 

Solution: In order to tidy this dataset, we need to <em>melt</em>. The pandas library has a built-in function that allows to do just that:<br> It “unpivots” a DataFrame from a <em>wide</em> format to a <em>long</em> format. 

In [3]:
formatted_df = pd.melt(df,
                       id_vars=["religion"],
                       var_name="income",
                       value_name="freq")
formatted_df.head()

,religion,income,freq
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Dont know/refused,<$10k,15


We can sort these results by religion

In [4]:
formatted_df = formatted_df.sort_values(by=["religion"])
formatted_df.head()

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34


Billboard Top 100 Dataset

This dataset represents the weekly rank of songs from the moment they enter the Billboard Top 100 to the subsequent 75 weeks.

Problems:
<ol>
<li>The columns headers are composed of values: the week number (x1st.week, …)</li>
<li>If a song is in the Top 100 for less than 75 weeks, the remaining columns are filled with missing values.</li>
</ol>

In [5]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2")
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A tidy version of this dataset is one without the week’s numbers as columns but rather as values of a single column. 

Solution: In order to do so, we’ll melt the weeks columns into a single date column. We will create one row per week for each record. If there is no data for the given week, we will not create a row.

Melting

Since we have a long list of id_vars, we'll create a list to hold the variables.


In [6]:
id_vars = ["year",
           "artist.inverted",
           "track",
           "time",
           "genre",
           "date.entered",
           "date.peaked"]



In [7]:
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")
df.tail()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
24087,2000,Ghostface Killah,Cherchez LaGhost,3:04,R&B,2000-08-05,2000-08-05,x76th.week,NaN
24088,2000,"Smith, Will",Freakin' It,3:58,Rap,2000-02-12,2000-02-12,x76th.week,NaN
24089,2000,Zombie Nation,Kernkraft 400,3:30,Rock,2000-09-02,2000-09-02,x76th.week,NaN
24090,2000,"Eastsidaz, The",Got Beef,3:58,Rap,2000-07-01,2000-07-01,x76th.week,NaN
24091,2000,Fragma,Toca's Miracle,3:22,R&B,2000-10-28,2000-10-28,x76th.week,NaN


Cleaning out unnecessary rows


In [8]:
df = df.dropna()
df.tail()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
19663,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,x63rd.week,45.0
19700,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,x63rd.week,50.0
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,x64th.week,50.0
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,x64th.week,50.0
20334,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,x65th.week,49.0


Formatting

We'll clean up the week column so only a number is left (i.e., remove x and rd.week)
We'll remove the decimal point in rank

In [9]:
df["week"] = df["week"].str.extract("(\d+)", expand=False).astype(int)
df["rank"] = df["rank"].astype(int)
df.tail()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
19663,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,63,45
19700,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,63,50
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,64,50
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,64,50
20334,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,65,49


Create "date" columns

In [10]:
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)
df.tail()


,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
19663,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,63,45,2000-08-12
19700,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,63,50,2000-11-18
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,64,50,2000-08-19
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,64,50,2000-11-25
20334,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,65,49,2000-12-02


Drop the columns: date.entered and date.peaked

Then sort table by year, artist, track, week, rank

In [11]:
df = df[["year", 
         "artist.inverted",
         "track",
         "time",
         "genre",
         "week",
         "rank",
         "date"]]
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])
df.tail()

,year,artist.inverted,track,time,genre,week,rank,date
10793,2000,matchbox twenty,Bent,4:12,Rock,35,33,2000-12-23
11110,2000,matchbox twenty,Bent,4:12,Rock,36,37,2000-12-30
11427,2000,matchbox twenty,Bent,4:12,Rock,37,38,2001-01-06
11744,2000,matchbox twenty,Bent,4:12,Rock,38,38,2001-01-13
12061,2000,matchbox twenty,Bent,4:12,Rock,39,48,2001-01-20


Assigning the tidy dataset to a variable for future usage

In [12]:
billboard = df
billboard.tail()

,year,artist.inverted,track,time,genre,week,rank,date
10793,2000,matchbox twenty,Bent,4:12,Rock,35,33,2000-12-23
11110,2000,matchbox twenty,Bent,4:12,Rock,36,37,2000-12-30
11427,2000,matchbox twenty,Bent,4:12,Rock,37,38,2001-01-06
11744,2000,matchbox twenty,Bent,4:12,Rock,38,38,2001-01-13
12061,2000,matchbox twenty,Bent,4:12,Rock,39,48,2001-01-20


A tidier version of the dataset is shown above. 

But...

There is still a lot of repetition of the song details: the track name, time and genre. For this reason, this dataset is still not completely tidy. We will address this in the next example.

Multiple types in one table

Following up on the Billboard dataset, we’ll now address the repetition problem of the previous table.

Problem: Multiple observational units (the song and its rank) in a single table.



.drop_duplicates() removes duplicate values in a column

In [13]:
billboard[["year", "artist.inverted", "track", "time", "genre"]].drop_duplicates().tail()

,year,artist.inverted,track,time,genre
254,2000,Yankee Grey,Another Nine Minutes,3:10,Country
277,2000,"Yearwood, Trisha",Real Live Woman,3:55,Country
251,2000,Ying Yang Twins,Whistle While You Twurk,4:19,Rap
314,2000,Zombie Nation,Kernkraft 400,3:30,Rock
15,2000,matchbox twenty,Bent,4:12,Rock


Using drop_duplicates, we'll create a songs table which contains the details of each song:

In [14]:
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]
songs = billboard[songs_cols].drop_duplicates()
songs.head()

,year,artist.inverted,track,time,genre
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B
24,2000,3 Doors Down,Kryptonite,3:53,Rock
193,2000,3 Doors Down,Loser,4:24,Rock
69,2000,504 Boyz,Wobble Wobble,3:35,Rap


The index is just a set of randomly ordered numbers.  Let's change that by resetting the index and creating a column called song_id. 

In [15]:
songs = songs.reset_index(drop=True)
songs["song_id"] = songs.index
songs.head()

,year,artist.inverted,track,time,genre,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1
2,2000,3 Doors Down,Kryptonite,3:53,Rock,2
3,2000,3 Doors Down,Loser,4:24,Rock,3
4,2000,504 Boyz,Wobble Wobble,3:35,Rap,4


We’ll then create a ranks table which only contains the song_id, date and the rank.

In [16]:
ranks = pd.merge(billboard, songs, on=["year","artist.inverted", "track", "time", "genre"])
ranks.head()

,year,artist.inverted,track,time,genre,week,rank,date,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26,0
1,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04,0
2,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11,0
3,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18,0
4,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25,0


In [17]:
ranks = ranks[["song_id", "date","rank"]]
ranks.head()

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87


We can look at how the rank of a particular song changes from week to week using the following syntax

In [18]:
ranks[ranks["song_id"] == 0]

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87
5,0,2000-04-01,94
6,0,2000-04-08,99


In [19]:
ranks[ranks["song_id"] == 0]["rank"].describe()

count     7.000000
mean     85.428571
std       9.360505
min      72.000000
25%      79.500000
50%      87.000000
75%      90.500000
max      99.000000
Name: rank, dtype: float64

# 2. Multiple variables stored in one column

Tubercolosis Records from World Health Organization

This dataset documents the count of confirmed tuberculosis cases by country, year, age and sex.

Problems:
<ol>
<li>Some columns contain multiple values: sex and age.  Note: m = male, f= female and numbers following m or f are age range</li>
<li>Mixture of zeros and missing values NaN. This is due to the data collection process and the distinction is important for this dataset.</li>
</ol>

In [20]:
df = pd.read_csv("./data/tb-raw.csv")
df.head()

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0


Solution:
In order to tidy this dataset, we need to remove the different values from the header and unpivot them into rows. We’ll first need to melt the sex + age group columns into a single one. Once we have that single column, we’ll derive three columns from it: sex, age_lower and age_upper. With those, we’ll be able to properly build a tidy dataset.

In [21]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


 Extract Sex, Age lower bound and Age upper bound group

In [22]:

tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})")    
tmp_df.head()


,0,1,2
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14


Create names for columns.

Then create `age` column based on `age_lower` and `age_upper`




In [23]:
tmp_df.columns = ["sex", "age_lower", "age_upper"]
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]
tmp_df.head()

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14
2,m,0,14,0-14
3,m,0,14,0-14
4,m,0,14,0-14


Merge df with tmp_df to create a tidier df

In [24]:
df.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


In [25]:

df = pd.concat([df, tmp_df], axis=1)
df.head()


,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


 Drop unnecessary columns and rows

In [26]:
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
df = df.dropna()
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head()

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54


# 3. Variables are stored in both rows and columns

Global Historical Climatology Network Dataset

This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010.

Problems:
Variables are stored in both rows (tmin, tmax) and columns (days).

In [27]:
df = pd.read_csv("./data/weather-raw.csv")
df

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Solution:

In order to make this dataset tidy, we want to move the three misplaced variables (tmin, tmax and days) as three individual columns: tmin. tmax and date.

Melting and Extracting day

In [28]:
id_vars=["id",
         "year",
        "month",
        "element"]

df = pd.melt(df, id_vars=id_vars, value_name="temp", var_name="day_raw")
df.head()

,id,year,month,element,day_raw,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [29]:
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  
df.head()

,id,year,month,element,day_raw,temp,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1


Convert to numeric values (make sure not strings)

Make sure id column is a string

In [30]:

df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))
df["id"] = "MX17004"
df.head()

,id,year,month,element,day_raw,temp,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1


Creating a single date from the year, month, day columns

Drop any entries that don't have a temperature measurment


In [31]:
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

df["date"] = df.apply(lambda row: create_date_from_year_month_day(row), axis=1)
df = df.drop(['year',"month","day", "day_raw"], axis=1)
df = df.dropna()
df

,id,element,temp,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02
22,MX17004,tmax,24.1,2010-02-03
23,MX17004,tmin,14.4,2010-02-03
44,MX17004,tmax,32.1,2010-03-05
45,MX17004,tmin,14.2,2010-03-05


Create a pivot table


In [32]:
df = pd.pivot_table(df, index=["id","date"], columns=["element"], values="temp")
df.reset_index(drop=False, inplace=True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


# 4. One observational unit stored in multiple tables

Dataset: Illinois Male Baby Names for the year 2014/2015.

Problems:
<ol>
<li>The data is spread across multiple tables/files.</li>
<li>The “Year” variable is present in the file name.</li>
<li>In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. </li>
<li>Furthermore, we’ll need to extract the “Year” variable from the file name.</li>
</ol>

Let's look at the data quickly

In [33]:
df = pd.read_csv("./data/2015-baby-names-illinois.csv")
df.head()

,rank,name,frequency,sex
0,1,Noah,863,Male
1,2,Liam,709,Male
2,3,Alexander,703,Male
3,4,Jacob,650,Male
4,5,William,618,Male


Custom script defined to grab the year from the file name

In [34]:
def extract_year(string):
    match = re.match(".+(\d{4})", string) 
    if match != None: return match.group(1)
    

Setting up variables to tell Python where to find our files

In [35]:
path = './data'
allFiles = glob.glob(path + "/201*-baby-names-illinois.csv")
frame = pd.DataFrame()
df_list= []


Grab both files from 2014 and 2015 and put into a DataFrame

In [36]:
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    df["year"] = extract_year(file_)
    df_list.append(df)
    
df = pd.concat(df_list)
df.head()

,rank,name,frequency,sex,year
0,1,Noah,863,Male,2015
1,2,Liam,709,Male,2015
2,3,Alexander,703,Male,2015
3,4,Jacob,650,Male,2015
4,5,William,618,Male,2015


<img src=images/parrotgiphy.gif width=300px>

<center> <b>Congratulations!  You finished Lesson 3.</b></center>

Link to [Lesson 4](04_Visualizing_Your_DataFrames.ipynb)